In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import requests
from datetime import date
import yfinance as yf

In [7]:
tick = "DIS"

In [74]:
def get_by_tick(tick):
    df = pd.read_csv("companies.csv")
    row = df[(df == tick).any(axis=1)]
    return row.values[0]

In [75]:
ticker, short, name = get_by_tick(tick)
print(ticker, short, name)

DIS disney The Walt Disney Company


In [76]:
def get_page_content(link):
    page = requests.get(link, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    content = soup(page.content, "html.parser")
    return content

In [77]:
def get_current_cashflow(ticker, short, precovid=True):
    URL = f"https://www.macrotrends.net/stocks/charts/{ticker}/{short}/free-cash-flow"
    page = get_page_content(URL)
    cashflows = {}

    tbody = page.find("table", "historical_data_table").find("tbody")
    for tr in reversed(tbody.findAll("tr")):
        trs = tr.findAll("td")
        year = int(trs[0].text.split("-")[0])
        if precovid and year > 2018: continue # eliminate post-covid
        price = float(trs[1].text.replace(",","").strip())
        cashflows[year] = price
    return cashflows

In [78]:
cashflows = get_current_cashflow(ticker, short, precovid=True)
cashflows

{2009: 3566.0,
 2010: 4468.0,
 2011: 3435.0,
 2012: 4182.0,
 2013: 6656.0,
 2014: 6469.0,
 2015: 7120.0,
 2016: 8363.0,
 2017: 8720.0,
 2018: 9830.0}

In [79]:
def get_growth_rates(cashflows):
    df = pd.DataFrame([cashflows])
    growth_rates = {}

    for year1, year2 in zip(df, df.iloc[:, 1:]):
        price1 = df[year1].iloc[0]
        price2 = df[year2].iloc[0]
        growth_rate = (price2 - price1) / price1
        growth_rates[year2] = round(growth_rate, 4)
    avg_growth_rate = round(np.mean(list(growth_rates.values())), 4)
    return growth_rates, avg_growth_rate

In [80]:
growth_rates, avg_growth_rate = get_growth_rates(cashflows)
growth_rates, avg_growth_rate

({2010: 0.2529,
  2011: -0.2312,
  2012: 0.2175,
  2013: 0.5916,
  2014: -0.0281,
  2015: 0.1006,
  2016: 0.1746,
  2017: 0.0427,
  2018: 0.1273},
 0.1387)

In [81]:
def get_future_cashflows(cashflows, avg_growth_rate):
    future_cashflows = {}
    year = date.today().year + 1
    future_cashflows[year] = np.floor(cashflows[list(cashflows.keys())[-1]] * (1 + avg_growth_rate))
    for year in range(year + 1, 2032):
        future_cashflows[year] = np.floor(future_cashflows[year-1] + (future_cashflows[year-1] * avg_growth_rate))

    return future_cashflows

In [82]:
future_cashflows = get_future_cashflows(cashflows, avg_growth_rate)
future_cashflows

{2023: 11193.0,
 2024: 12745.0,
 2025: 14512.0,
 2026: 16524.0,
 2027: 18815.0,
 2028: 21424.0,
 2029: 24395.0,
 2030: 27778.0,
 2031: 31630.0}

In [83]:
# rate at which the company will grow in the future. Usually 2.5%-3% as economy moves in that pace
perpetual_growth_rate = 0.025
# how much cheaper we want it to get
dicount_rate = 0.08

In [84]:
def get_terminal_value(future_cashflows, perpetual_growth_rate, dicount_rate):
    price_2030 = future_cashflows[list(future_cashflows.keys())[-1]]
    terminal_value = np.floor(price_2030 * (1+perpetual_growth_rate) / (dicount_rate-perpetual_growth_rate))
    future_cashflows["Terminal"] = terminal_value
    return future_cashflows

In [85]:
future_cashflows = get_terminal_value(future_cashflows, perpetual_growth_rate, dicount_rate)
future_cashflows

{2023: 11193.0,
 2024: 12745.0,
 2025: 14512.0,
 2026: 16524.0,
 2027: 18815.0,
 2028: 21424.0,
 2029: 24395.0,
 2030: 27778.0,
 2031: 31630.0,
 'Terminal': 589468.0}

In [86]:
def get_PV_of_FFCFs(future_cashflows, dicount_rate):
    PV_of_FFCFs = {}
    for (i, key) in enumerate(future_cashflows.keys()):
        PV_of_FFCFs[key] = np.floor(future_cashflows[key] / (1 + dicount_rate)**(i+1))
    sum_of_FFCFs = np.sum(list(PV_of_FFCFs.values()))
    return PV_of_FFCFs, sum_of_FFCFs

In [87]:
PV_of_FFCFs, sum_of_FCFs = get_PV_of_FFCFs(future_cashflows, dicount_rate)
PV_of_FFCFs, sum_of_FCFs

({2023: 10363.0,
  2024: 10926.0,
  2025: 11520.0,
  2026: 12145.0,
  2027: 12805.0,
  2028: 13500.0,
  2029: 14234.0,
  2030: 15007.0,
  2031: 15822.0,
  'Terminal': 273037.0},
 389359.0)

In [138]:
def convert_to_B(val):
    if (val[-1] == "B"):
        val = val.replace("B", "")
        while (len(val.split(".")[-1]) < 3):
            val += "0"
        val = int(val.replace(".", ""))
        return val
    elif (val[-1] == "M"):
        val = val.replace(".", "").replace("M", "")
        val = float("0." + val)
        return val

In [97]:
def get_cash_and_eq(tick):
    return yf.Ticker(tick).balance_sheet.loc["Cash"][0]/1_000_000

In [98]:
cash_and_eq = get_cash_and_eq(tick)
cash_and_eq

15959.0

In [109]:
def get_total_debt_and_shares(tick):
    total_debt = int(yf.Ticker(tick).info["totalDebt"]/1_000_000)
    shares_outstanding = int(yf.Ticker(tick).info['sharesOutstanding']/1_000_000)

    return total_debt, shares_outstanding

In [110]:
total_debt, shares_outstanding= get_total_debt_and_shares(tick)
total_debt, shares_outstanding

(52023, 1821)

In [111]:
def get_equity_and_intrinsic(sum_of_FCF, cash_and_eq, total_debt, shares_outstanding):
    equity_value = sum_of_FCF + cash_and_eq - total_debt
    
    price_per_share = round(equity_value / shares_outstanding, 2)
    
    return equity_value, price_per_share

In [112]:
equity_value, price_per_share = get_equity_and_intrinsic(sum_of_FCFs, cash_and_eq, total_debt, shares_outstanding)
equity_value, price_per_share

(353295.0, 194.01)